<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Test comparing the original OverlandFlow Landlab component to our updated version of OverlandFlow
This notebook was created by Sam Anderson, Mikey Sison and Angel Monsalve.

<hr>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

** What is this notebook? **

This notebook generates results to compare the behaivor of the overland flow model presented by Adams et al. (2017) to the updated version used here to eventually couple with a sediment transport component. 

Our version of OverlandFlow was tested using the analytical solution for wave propagation on a flat surface case as first presented by Adams et al. (2017) and originally developed by Bates et al. (2010) and Hunter et al. (2005).
The idea of this test is to check if our version behaves in the same manner thand as the original OverlandFlow does under unsteady flow conditions. Our simulations were conducted in a flat domain of 6000 m long (x direction) 
and 800 m high (y direction) in which we used three different uniform grid configurations (∆x=∆y) with spacings of ∆x=5, 10, and 50 m. At the beginning of the simulation the domain was filled with a thin film of water of 1 mm. 
A uniform roughness coefficient was used (n=0.03). Boundary conditions at the top, right, and bottom edges of the domain were set to closed. Flow enters through the left edge (x=0 m) with a constant velocity of u=1 m/s and v=0 
m/s and water depth changes in time according to h(x,t)=h(0,t)=(7⁄3 n^2 u^3 t)^(3⁄7). 

After 3600 s of simulation we obtained identical results to those presented by Adams et al. (2017), indicating that our modifications to OverlandFlow do not affect the core of the flow solver (Figure 6). A detailed validation
is out of the scope our test and analysis but for more details on the validation the reader is referred to Adams et al. (2017).

More background on the model used here and the results presented will be published in the future and the citation will be added here:

**TITLE HERE**

The code used in this exercise is taken from the above reference.



**Now on to the code.**
* Below we import Landlab components, functions for importing data, numpy and plotting tools. You should not need to change this.

In [1]:
## Code Block 1

%reset -f
import numpy as np
import copy
from matplotlib import pyplot as plt
from landlab.components import OverlandFlowSpatiallyVariableInputs
from landlab.io import read_esri_ascii
from landlab import imshow_grid



Now we import the data for the watershed we want to route flow on, as well as some model parameters 

In [2]:
## Code Block 2

bedElevationDEM = 'bedElevationDEM_dx5.asc'           # ASCII raster DEM containing the bed surface elevation
n = 0.03        # Manning's n
u = 1.0         # Constant velocity at the left edge flowing in the positive x direction - Boundary condition
dtPrecision = 3 # Avoids rounding errors
tPlot = 3600     # Plots will be obtained every amount of time in seconds specified here #VALUE CHANGED
storeData = 600 # Stores results every amount of time in seconds specified here
tmax = 3600     # Maximum simulation time [s]
hMax = 3.0      # Maximum height (in meters) expected - Used only for plots
h_init = 0.001  # Initial water depth in meters.


In [3]:
## Code Block 3

# Reads all rasters containing bed elevation, roughness, GSD location, and Rainfall intensity
(rmg, z) = read_esri_ascii(bedElevationDEM, name='topographic__elevation') 
rmg['link']['bed_surface__roughness'] = np.zeros(rmg.number_of_links) + n
rmg['node']['rainfall_intensity'] = np.zeros(rmg.number_of_nodes) 

In [4]:
## Code Block 4

OverlandFlowSpatiallyVariableInputs.input_var_names #Inputs necessary variable names

rmg.add_zeros('surface_water__depth', at = 'node') # Creates all topographic, sediment, and flow variables

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [5]:
## Code Block 6

# Sets boundaries to closed 
rmg.set_closed_boundaries_at_grid_edges(True,True,True,True)  


In [6]:
## Code Block 7

# Instantiation according to Adams et al 2017
of = OverlandFlowSpatiallyVariableInputs(rmg, h_init = h_init, steep_slopes = False, theta = 1.0, alpha = 0.7, dt_max = 7.5)

In [7]:
## Code Block 8

# ID for boundary nodes
# First we need to generate a list with the west-east nodes ID next to the left edge
# For a 120 columns raster these will be 121 241 361 etc..
nodesBoundary_Id = np.arange(rmg.number_of_node_columns,rmg.number_of_node_columns*(rmg.number_of_node_rows-1),(rmg.number_of_node_columns))

# ID for boundary links
# Now we need to generate a list with the west-east links ID at the left edge
# For a 120 column raster these will be 240 479 718 etc..
linksBoundary_Id = np.arange((2*rmg.number_of_node_columns)-1,rmg.number_of_links,(2*rmg.number_of_node_columns)-1)

In [8]:
## Code Block 9

# Defines some variables used to store data
t = 0                                   # Initializates the variable
storeDataNow = True                     # Used to save the plot at time zero and tmax
plotNow = True                          # Used to save the plot at time zero and tmax
check_tmax = True
tPlotOrg=copy.deepcopy(tPlot)           # A copy of tPlot
storeDataOrg=copy.deepcopy(storeData)   # A copy of tPlot

In [9]:
## Code Block 10

while t <= tmax:
    
    h_boundary = np.max((0.001,(7/3 * n**2 * u**3 * t) ** (3/7)))
    q_boundary = u * h_boundary
    
    # Updates data in the grid
    rmg["node"]["surface_water__depth"][nodesBoundary_Id] = h_boundary
    rmg["link"]["surface_water__depth"][linksBoundary_Id] = h_boundary
    rmg["link"]["surface_water__discharge"][linksBoundary_Id] = q_boundary
    of.overland_flow()  # Runs overland flow for one time step
    
    ## Stores results
    storeData = round(storeData-of.dt, dtPrecision)
    if (storeData <= 0) or storeDataNow:
        print('Storing results at time :',np.round(t,1),' s')      
        data = np.reshape(np.hstack([t,(of._h.T)]),[1,rmg.number_of_nodes+1])
        with open("output1_node_surface_water__depth.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')              
        storeData = round(storeDataOrg, dtPrecision)
        storeDataNow = False

    tPlot = round(tPlot-of.dt, dtPrecision)
    if tPlot <= 0  or plotNow:
        print('Elapsed time :',np.round(t,1),' s. Current dt =',\
              np.round(of.dt,1),'. Adaptive time =',np.round(of._adaptive_dt,1),' s - Saving plot')
        
        """ Water depth plot"""
        plot_name='Surface water depth [m] at ' + str(np.round(t,0)) + ' sec'
        imshow_grid(rmg, 'surface_water__depth',cmap='Blues',vmin=0,vmax=hMax,plot_name=plot_name)
        output='depth_'+str(np.round(t,0))+'.png'
        plt.savefig(output,dpi=300); plt.close()   
               
        plotNow = False
        tPlot = tPlotOrg
        
            ## Updating t
    if (t + of.dt > tmax) and check_tmax:
        of.dt = tmax - t
        t = tmax
        storeDataNow = True  
        plotNow = True
        check_tmax = False
    else:
        t = round(t + of.dt, dtPrecision)

Storing results at time : 0  s
Elapsed time : 0  s. Current dt = 7.5 . Adaptive time = 35.3  s - Saving plot
Storing results at time : 606.5  s
Storing results at time : 1207.5  s
Storing results at time : 1808.2  s
Storing results at time : 2408.4  s
Storing results at time : 3008.8  s
Storing results at time : 3600  s
Elapsed time : 3600  s. Current dt = 0.7 . Adaptive time = 0.7  s - Saving plot


In [10]:
## Code Block 11

# Results
## Water depth at the end of the simulation
data = np.loadtxt('output1_node_surface_water__depth.txt')
nodesToSample = np.arange(rmg.number_of_node_columns+1,2*rmg.number_of_node_columns)
x = np.arange(0,(rmg.number_of_node_columns-1)*rmg.dx,rmg.dx)

hSample = data[-1,nodesToSample]

hAnalytical = np.zeros_like(x)+h_init
hAnalytical[np.where(x<u*tmax)] =(-7/3 * (n**2 * u**2 * (x[np.where(x<u*tmax)]-u*tmax)))**(3/7)

plt.figure(1)
plt.plot(x, hSample, color='mediumblue')
plt.plot(x, hAnalytical, color='black')
plt.ylabel('Water depth (m)')
plt.xlabel('x (x)')
plt.ylim(0,2.5)
plt.xlim(0,5000)
plt.title('Water depth at 3600 s')
plt.savefig('WaterDepth.png',dpi=300);plt.close()
